Programas de línea de comandos

## Questions

- ¿Cómo puedo escribir programas Python que funcionen como herramientas de línea de comandos Unix?

## Objetivos

- Utilice los valores de los argumentos de línea de comandos en un programa.

- Manejar banderas y archivos por separado en un programa de línea de comandos.

El Jupyter Notebook y otras herramientas interactivas son excelentes para crear prototipos de código y explorar datos, pero tarde o temprano vamos a querer usar nuestro programa en una tubería o ejecutarlo en un script por lotes para procesar miles de archivos de datos. Para hacer eso, necesitamos hacer que nuestros programas funcionen como otras herramientas de línea de comandos (o incluso darle una interfaz gráfica de usuario - GUI - pero eso está más allá del alcance de este curso). Por ejemplo, es posible que queramos un programa que lea un conjunto de datos de luminancia e imprima la media en Cd/m 2.


Este programa hace exactamente lo que queremos - imprime la luminancia promedio para un conjunto de datos dado en el rango especificado.

###### Command Line
```
> python ./luminance_processor.py --mean simulation01.txt 
```
###### Output
```
343000
```
(actual values may differ)

También podríamos querer ver los valores de luminancia en varios archivos uno tras otro:
###### Command Line

```
> python ./luminance_processor.py --mean simulation01.txt simulation02.txt 
```
Nuestros scripts deben hacer lo siguiente:

1. Si se dan uno o varios nombres de archivo, se leerán los datos de los mismos y se presentarán estadísticas para cada archivo por separado.
2. Utilice la media o la STD para determinar qué estadísticas imprimir.
3. Utilizar un indicador -experimento para indicar que deseamos comparar la simulación dada con un archivo de experimento dado

Para que esto funcione, necesitamos saber cómo manejar argumentos de línea de comandos en un programa.

# Command-Line Arguments

Usando el editor de texto de su elección (*e.g.* Notepad++ o VSCode), guarde lo siguiente en un archivo de texto llamado `sys_version.py`:

```Python
import sys
print('version is', sys.version)
```

La primera línea importa una biblioteca llamada `sys`, que es la abreviatura de "sistema". Define valores como `sys.versión`, que describe qué versión de Python estamos ejecutando. Podemos ejecutar este script desde la línea de comandos así:

###### Command Line
```
> python sys_version.py
```

###### Output
```
version is 3.4.3+ (default, Jul 28 2015, 13:17:50)
[GCC 4.9.3]
```

Cree otro archivo llamado `argv_list.py` y guarde el siguiente texto en él.

```Python
import sys
print('sys.argv is', sys.argv)
```

El extraño nombre `argv` significa "valores de argumento". Cada vez que Python ejecuta un programa, toma todos los valores dados en la línea de comandos y los pone en la lista `sys.argv` para que el programa pueda determinar lo que eran. Si ejecutamos este programa sin argumentos:

###### Command Line
```
> python argv_list.py
```

###### Output
```
sys.argv is ['argv_list.py']
```

la única cosa en la lista es la ruta completa a nuestro script, que es siempre `sys.argv[0]`. Si lo ejecutamos con algunos argumentos, sin embargo:

###### Command Line
```
> python argv_list.py first second third
```

###### Output
```
sys.argv is ['argv_list.py', 'first', 'second', 'third']
```

luego Python agrega cada uno de esos argumentos a esa lista mágica.

Con esto en la mano, vamos a construir una versión de `luminance_processor.py` que siempre imprime la media de luminancia de un solo archivo de datos en el rango de -10 mm a 10 mm. El primer paso es escribir una función que describa nuestra implementación, y un marcador de posición para la función que hace el trabajo real. Por convención esta función suele llamarse `main`, aunque podemos llamarla como queramos. Llamemos a este nuevo script `luminance.py`. En lugar de volver a escribir el código de lecciones anteriores, vamos a copiar/ pegar las dos funciones que hemos utilizado en las lecciones anteriores: `find_data_cross_section` y `calculate_average_luminance`.

```Python
import sys
import numpy as np


def find_data_cross_section(simulation_filename):
    simulation = np.loadtxt(fname=simulation_filename, skiprows=52)

    x_simulation = simulation[:, 0]
    y_simulation = simulation[:, 1]
    L_simulation = simulation[:, 2]
    smallest_y = np.amin(abs(y_simulation))
    x_cross_section = x_simulation[y_simulation==smallest_y]
    luminance_cross_section = L_simulation[y_simulation==smallest_y]
    
    return x_cross_section, luminance_cross_section


def calculate_average_luminance(x, luminance):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    average_luminance = np.mean(luminance[boolean_array])
    return average_luminance


def main():
    filename = sys.argv[1]
    x, luminance = find_data_cross_section(filename)
    mean = calculate_average_luminance(x, luminance)
    print('Average luminance for ' + filename + ' = ' + str(mean) + ' Cd/m^2')
```

Here’s a simple test:

```
python ./luminance.py simulation01.txt
```

No hay salida porque hemos definido una función pero no la hemos llamado. Agreguemos una llamada a main:

```Python
import sys
import numpy as np


def find_data_cross_section(simulation_filename):
    simulation = np.loadtxt(fname=simulation_filename, skiprows=52)

    x_simulation = simulation[:, 0]
    y_simulation = simulation[:, 1]
    L_simulation = simulation[:, 2]
    smallest_y = np.amin(abs(y_simulation))
    x_cross_section = x_simulation[y_simulation==smallest_y]
    luminance_cross_section = L_simulation[y_simulation==smallest_y]
    
    return x_cross_section, luminance_cross_section


def calculate_average_luminance(x, luminance):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    average_luminance = np.mean(luminance[boolean_array])
    return average_luminance


def main():
    filename = sys.argv[1]
    x, luminance = find_data_cross_section(filename)
    mean = calculate_average_luminance(x, luminance)
    print('Average luminance for ' + filename + ' = ' + str(mean) + ' Cd/m^2')
    
    
if __name__ == '__main__':
   main()
```

and run that:

###### Command Line
```
> python ./luminance.py simulation01.txt
```
###### Output
```
Average luminance for simulation01.txt = 315570.128125 Cd/m^2
```

> **Ejecutar vs Importar:** Ejecutar un script Python en la línea de comandos es muy similar a importar ese archivo en Python. La mayor diferencia es que no esperamos que suceda nada cuando importamos un archivo, mientras que cuando ejecutamos un script, esperamos ver algunos resultados impresos en la consola.
>
> Para que un script Python funcione como se espera cuando se importa o cuando se ejecuta como un script, normalmente ponemos la parte del script que produce la salida en la siguiente instrucción if:
> ```Python
> if __name__ == '__main__':
>     main()  # Or whatever function produces output
> ```
> Cuando importa un archivo Python, `_name_` se establece en el nombre de ese archivo (por ejemplo, al importar readings.py, `_name_` es 'readings'). Sin embargo, cuando ejecuta un script en la línea de comandos, `_name__` siempre se establece en `'_main__'` en ese script para que pueda determinar si el archivo está siendo importado o ejecutado como un script.

# La forma "correcta" de hacerlo

Si nuestros programas pueden tomar parámetros complejos o múltiples nombres de archivo, no debemos manejar `sys.argv` directamente. En su lugar, debemos usar la biblioteca `argparse` de Python, que maneja casos comunes de manera sistemática, y también nos facilita proporcionar mensajes de error sensatos para nuestros usuarios. No cubriremos este módulo en esta lección, pero usted puede ir a Tshepang Lekhonkhobe [argparse tutorial](http://docs.python.org/3/howto/argparse.html) que es parte de la documentación oficial de Python.

# Manejo de múltiples archivos

El siguiente paso es enseñar a nuestro programa cómo manejar varios archivos. 

Queremos que nuestro programa procese cada archivo por separado, así que necesitamos un bucle que se ejecute una vez por cada nombre de archivo. Si especificamos los archivos en la línea de comandos, los nombres de archivo estarán en `sys.argv`, pero debemos tener cuidado: `sys.argv[0]` siempre será el nombre de nuestro script, en lugar del nombre de un archivo. También necesitamos manejar un número desconocido de nombres de archivo, ya que nuestro programa podría ejecutarse para cualquier número de archivos.

La solución a ambos problemas es hacer un bucle sobre el contenido de `sys.argv[1:]`. El '1' le dice a Python que inicie la rebanada en la ubicación 1, por lo que el nombre del programa no está incluido; ya que hemos dejado el límite superior, la rebanada se ejecuta hasta el final de la lista, e incluye todos los nombres de archivo. Aquí está nuestro programa cambiado:

```Python
import sys
import numpy as np


def find_data_cross_section(simulation_filename):
    simulation = np.loadtxt(fname=simulation_filename, skiprows=52)

    x_simulation = simulation[:, 0]
    y_simulation = simulation[:, 1]
    L_simulation = simulation[:, 2]
    smallest_y = np.amin(abs(y_simulation))
    x_cross_section = x_simulation[y_simulation==smallest_y]
    luminance_cross_section = L_simulation[y_simulation==smallest_y]
    
    return x_cross_section, luminance_cross_section


def calculate_average_luminance(x, luminance):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    average_luminance = np.mean(luminance[boolean_array])
    return average_luminance


def main():
    filenames = sys.argv[1:]
    for filename in filenames:
        x, luminance = find_data_cross_section(filename)
        mean = calculate_average_luminance(x, luminance)
        print('Average luminance for ' + filename + ' = ' + str(mean) + ' Cd/m^2')
    
    
if __name__ == '__main__':
   main()
```
and here it is in action:

###### Command Line
```
> python ./luminance.py simulation01.txt simulation02.txt
```

###### Output
```
Average luminance for simulation01.txt = 315570.128125 Cd/m^2
Average luminance for simulation02.txt = 67531.74755859375 Cd/m^2
```


# Handling Command-Line Flags

The next step is to teach our program to pay attention to the `--mean`, and `--std` flags. These always appear before the names of the files, so we can do this:

```Python
import sys
import numpy as np


def find_data_cross_section(simulation_filename):
    simulation = np.loadtxt(fname=simulation_filename, skiprows=52)

    x_simulation = simulation[:, 0]
    y_simulation = simulation[:, 1]
    L_simulation = simulation[:, 2]
    smallest_y = np.amin(abs(y_simulation))
    x_cross_section = x_simulation[y_simulation==smallest_y]
    luminance_cross_section = L_simulation[y_simulation==smallest_y]
    
    return x_cross_section, luminance_cross_section


def calculate_average_luminance(x, luminance):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    average_luminance = np.mean(luminance[boolean_array])
    return average_luminance


def calculate_std_luminance(x, luminance):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    std_luminance = np.std(luminance[boolean_array])
    return std_luminance


def main():
    action = sys.argv[1]
    filenames = sys.argv[2:]
    for filename in filenames:
        x, luminance = find_data_cross_section(filename)
        if action == '--mean':
            mean = calculate_average_luminance(x, luminance)
            print('Average luminance for ' + filename + ' = ' + str(mean) + ' Cd/m^2')
        elif action == '--std':
            std = calculate_std_luminance(x, luminance)
            print('Standard deviation on the luminance for ' + filename + ' = ' + str(std) + ' Cd/m^2')
        else:
            # this raises an error, and provides the message given as argument.
            raise TypeError('No valid action supplied (allowed actions are "--mean" or "--std"')

    
if __name__ == '__main__':
   main()
```
This works:

###### Command Line
```
> python .\luminance.py --std simulation01.txt simulation02.txt
```
###### Output
```
Standard deviation on the luminance for simulation01.txt = 25504.704660288367 Cd/m^2
Standard deviation on the luminance for simulation02.txt = 6609.0550659955 Cd/m^2
```

Bien, ya casi llegamos. Nuestra función `main()` se está poniendo un poco desordenada, además de que los cálculos de luminancia parecen tener mucho código extra. Simplifiquemos un poco nuestro código. Las siguientes dos funciones contienen código muy similar, ¿hay alguna forma de combinarlas?

```Python

# Let's take these two functions and combine them, by using a new argument
def calculate_average_luminance(x, luminance):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    average_luminance = np.mean(luminance[boolean_array])
    return average_luminance


def calculate_std_luminance(x, luminance):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    std_luminance = np.std(luminance[boolean_array])
    return std_luminance

```

As it happens, yes! We can pass the action to the function and depending on the value of `action` we can do one of the two different calculations we need.

```Python
# This new function also incorporates the conditional from the main loop as well
def calculate_luminance_stats(x, luminance, action):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    if action == '--mean':
        statistic = np.mean(luminance[boolean_array])
    elif action == '--std':
        statistic = np.std(luminance[boolean_array])
    else:
        raise TypeError('No valid action supplied (allowed actions are "--mean" or "--std")')
    return statistic

```

The code now looks like this:

```Python
import sys
import numpy as np


def find_data_cross_section(simulation_filename):
    simulation = np.loadtxt(fname=simulation_filename, skiprows=52)

    x_simulation = simulation[:, 0]
    y_simulation = simulation[:, 1]
    L_simulation = simulation[:, 2]
    smallest_y = np.amin(abs(y_simulation))
    x_cross_section = x_simulation[y_simulation==smallest_y]
    luminance_cross_section = L_simulation[y_simulation==smallest_y]
    
    return x_cross_section, luminance_cross_section


def calculate_luminance_stats(x, luminance, action):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    if action == '--mean':
        statistic = np.mean(luminance[boolean_array])
    elif action == '--std':
        statistic = np.std(luminance[boolean_array])
    else:
        raise TypeError('No valid action supplied (allowed actions are "--mean" or "--std")')
    return statistic


def main():
    action = sys.argv[1]
    filenames = sys.argv[2:]
    for filename in filenames:
        x, luminance = find_data_cross_section(filename)
        simulation_statistic = calculate_luminance_stats(x, luminance, action)
        print(action[2:] + ' luminance for ' + filename + ' = ' + str(simulation_statistic) + ' Cd/m^2')

    
if __name__ == '__main__':
   main()
```

Which is simpler than before! We can even create a dynamic print statement by slicing `action` .

Nuestro código está un poco más ordenado ahora. Finalmente, necesitamos incorporar una comparación opcional con datos experimentales.

Para hacer esto, vamos a introducir un emparejamiento "flag - value". El usuario debe proporcionar `-experiment filename` como un par para que funcione. Lo incorporaremos después de la primera acción y antes de los nombres de archivo.


```Python
import sys
import numpy as np


def find_data_cross_section(simulation_filename):
    simulation = np.loadtxt(fname=simulation_filename, skiprows=52)

    x_simulation = simulation[:, 0]
    y_simulation = simulation[:, 1]
    L_simulation = simulation[:, 2]
    smallest_y = np.amin(abs(y_simulation))
    x_cross_section = x_simulation[y_simulation==smallest_y]
    luminance_cross_section = L_simulation[y_simulation==smallest_y]
    
    return x_cross_section, luminance_cross_section


# we introduce a new function to load the experimental data
def load_experiment_data(experiment_filename):
    experiment = np.loadtxt(fname=experiment_filename, delimiter=',')
    return experiment[:,0], experiment[:,1]


def calculate_luminance_stats(x, luminance, action):
    boolean_array = np.logical_and(x<=10., x>= -10.)
    if action == '--mean':
        statistic = np.mean(luminance[boolean_array])
    elif action == '--std':
        statistic = np.std(luminance[boolean_array])
    else:
        raise TypeError('No valid action supplied (allowed actions are "--mean" or "--std")')
    return statistic


def main():
    action1 = sys.argv[1]
    action2 = sys.argv[2]
    
    if action2 == '--experiment':
        experiment_filename = sys.argv[3]
        x_experiment, luminance_experiment = load_experiment_data(experiment_filename)
        filenames = sys.argv[4:]
    else:
        experiment_filename = ''
        filenames = sys.argv[2:]
        
    for filename in filenames:
        x, luminance = find_data_cross_section(filename)
        simulation_statistic = calculate_luminance_stats(x, luminance, action1)
        print(action1[2:] + ' luminance for ' + filename + ' = ' + str(simulation_statistic) + ' Cd/m^2')
        if bool(experiment_filename):
            experiment_statistic = calculate_luminance_stats(x_experiment, luminance_experiment, action1)
            perc_diff = (experiment_statistic - simulation_statistic)*100/experiment_statistic
            print('Percentage difference with experimental value = ' + str(perc_diff) + ' %')

    
if __name__ == '__main__':
   main()
```

El código comprueba si `action2` es igual a `'-experimento'`. Si lo hace, entonces el siguiente valor se analiza como el nombre del archivo de experimento, de lo contrario el resto de los argumentos se supone que son nombres de archivo de simulación.

Luego, después de calcular las estadísticas, usamos `bool()` para comprobar si se ha pasado un archivo experimental. Si lo ha hecho, hacemos el cálculo experimental de la diferencia e imprimimos la diferencia.

Eso es mejor. De hecho, ya está hecho: el programa ahora hace todo lo que nos propusimos hacer.

# Exercise 1* - Arithmetic on the Command Line

Write a command-line program that does addition and subtraction:

###### Command Line
```
> python arith.py add 1 2
```
###### Output
```
3
```

###### Command Line
```
> python arith.py subtract 3 4
```
###### Output
```
-1
```

In [ ]:
# Create your solution in a text editor, but feel free to use this space to test things out!

# Ejercicio 2* - Encontrar archivos particulares

Usando el módulo `glob` introducido anteriormente, escriba una versión simple del comando unix `ls` que muestre archivos en el directorio actual con un sufijo particular. Una llamada a este script debería verse así:

###### Command Line
```
> python my_ls.py py
```
###### Output
```
left.py
right.py
zero.py
```

In [ ]:
# Create your solution in a text editor, but feel free to use this space to test things out!

# Ejercicio 3 - Agregar un mensaje de ayuda

Por separado, modifique `luminance.py` para que si no se dan parámetros (es decir, no se especifica ninguna acción y no se dan nombres de archivo), imprima un mensaje explicando cómo debe usarse.

In [2]:
# Create your solution in a text editor, but feel free to use this space to test things out!

# Ejercicio 4 - Agregar una acción por defecto

Por separado, modifique `luminance.py` para que si no se da ninguna acción muestre los medios de los datos.

In [2]:
# Create your solution in a text editor, but feel free to use this space to test things out!

# Ejercicio 5* - Generar un mensaje de error

Escribe un programa llamado `check_arguments.py` que imprime el uso previsto y luego sale del programa si no se proporcionan argumentos. (Sugerencia: Puede usar `sys.exit()` para salir del programa.)

###### Command Line
```
> python check_arguments.py
```
###### Output
```
usage: python check_argument.py filename.txt
```
###### Command Line
```
> python check_arguments.py filename.txt
```
###### Output
```
Thanks for specifying arguments!
```

In [ ]:
# Create your solution in a text editor, but feel free to use this space to test things out!

# Puntos clave

- La biblioteca `sys` conecta un programa Python al sistema en el que se ejecuta.

- La lista `sys.argv` contiene los argumentos de la línea de comandos con los que se ejecutó un programa.